<h1>Textbook RSA</h1>

In [1]:
from Crypto.Util.number import *
from Crypto import Random
import Crypto
import binascii
from xgcd import * 
from print_bytes_util import print_bytes

Task: Try different bit length and messages.

In [2]:
rsa_bit_length = 512
plain_text = 'Hello World'

<h2>RSA Key Generation</h2>

In [3]:
#Generate two large random primes, p and q, of approximately equal size 
p = Crypto.Util.number.getPrime(rsa_bit_length, randfunc=Crypto.Random.get_random_bytes)
q = Crypto.Util.number.getPrime(rsa_bit_length, randfunc=Crypto.Random.get_random_bytes)

#Calculate the modulus n:  n = p ∙ q
n = p*q

#Calculate 𝛗𝒏= (p 1) ∙(q 1)
PHI=(p-1)*(q-1)

#Choose public exponent e co prime to 𝛗𝒏and 𝐞≠±𝟏
e=65537

#Calculate secret exponent 𝒅=𝒆^−𝟏 (𝒎𝒐𝒅𝛗𝒏)
d= xgdc(e, PHI)

print(f'RSA key length = {rsa_bit_length} bits')
print(f'Message (plain text) = {plain_text}\n')

print('prime number p = 0x' + str(hex(p))[2:].upper() )
print('prime number q = 0x' + str(hex(q))[2:].upper() + '\n')

print('Public Key (e, n):')
print('  e = 0x' + str(hex(e))[2:].upper() )
print('  n = 0x' + str(hex(n))[2:].upper() + '\n')

print('Private Key (d, n):')
print('  d = 0x' + str(hex(d))[2:].upper() )
print('  n = 0x' + str(hex(n))[2:].upper() + '\n')


RSA key length = 512 bits
Message (plain text) = Hello World

prime number p = 0xD0B9B863DE14CF5B0D4537711B3AFC1BBBCC3AD87ADC7E9F2126D71D34830387BBED3583ECF4146C3FA829652BB7855BF027C15EE43EB06E6BC034312D4843FF
prime number q = 0x9BF7713CAA7D47EF675C19425B75916B74657AA9145925A3D674DC3C786BFBFFBED0169BEC7C1B0A1E0036AD3787A3BB834DAAB183E7CDC651BCEC1EED08D45B

Public Key (e, n):
  e = 0x10001
  n = 0x7F2A3228D0BEA5D77C546AC3243889B73D1B2AF907C7191CDBD4E8406848A956D0631CC31AE5A5D541E0F39EEA8863C3DD5F66D91E787A1508CA60D9C04BCCEA551A19A8ED4C186F3B8FC977A77321C6810912FBADA0417BD5FAF48623A133C114DE6B855B081502B6FC5423CB5147EAF987D6963A1DD3202C6BA41223F757A5

Private Key (d, n):
  d = 0x44BBAAF6F2B549706A8C3A54959C559B8C54C5EA6AB0CBBB147B44B9A3643ADB68B159977F4F474E38587D7594F0F973A190F24742B7FDCCA6BEB4909DE69AAFFD06D34EC61ADF896058FF3F93D76847BCCB5B842932991E66DB779FAFDC6C6C020A0B2D5359A5C2317A363EBE50A506D1BDDC83CE54703AFCEFC099BD9B7535
  n = 0x7F2A3228D0BEA5D77C546AC3243889B73D1B2AF907C7191CD

<h2>RSA Sign - Alice generates a RSA signature</h2>
Signing is using the private key {d, n}<br>
* Calculate the message hash: h = hash(M)<br>
* Sign h to calculate the signature: s = h^d (mod n)


In [4]:
from cryptography.hazmat.primitives import hashes

print('Message = ' + plain_text)
digest = hashes.Hash( hashes.SHA1() )
alice_baMessage = plain_text.encode('utf-8')
digest.update(alice_baMessage)
alice_hashFromMessage_bytes = digest.finalize()

print('Alice: h = hash(message) 0x' + alice_hashFromMessage_bytes.hex(' ').upper())

alice_hashFromMessage_long =  bytes_to_long(alice_hashFromMessage_bytes)
alice_rsa_signature = pow(alice_hashFromMessage_long, d, n)

print_bytes('Alice: RSA Signature s = 0x', long_to_bytes(alice_rsa_signature), 16)


Message = Hello World
Alice: h = hash(message) 0x0A 4D 55 A8 D7 78 E5 02 2F AB 70 19 77 C5 D8 40 BB C4 86 D0
Alice: RSA Signature s = 0x54 93 56 FF 88 B0 7B E6 D3 DC 7A CB F5 4F 3E 05 
                           19 D5 72 4F 7D 12 CE F3 5A 69 49 8E B2 A5 0B F5 
                           9E FB 79 33 D8 EA E7 21 C2 BC 0D FD E7 CC 4A 95 
                           4E C3 3A 62 17 1F C9 A4 68 BA 96 9C C0 AA 02 78 
                           F3 AC 8F ED 80 16 8A 21 F4 D6 98 36 CA 10 4C 07 
                           4B F8 95 27 68 A2 3B 1E 79 47 9F FB 09 AD BF CF 
                           54 60 5B FE 4C 99 A3 09 94 60 BF 3E 2A BA 09 B3 
                           73 2C 38 9C AF 31 DA FB 1B 4A 8B 85 30 EB C8 03 
                           


<h2>RSA Verify- Bob performs the signature verification</h2>
Verify is using the public key {d, n}<br>
* Calculate the message hash: h = hash(M)<br>
* Verify the signature: h' = s^e (mod n)<br>
* Compare h with h' to check if the signature is valid or not

In [5]:
from cryptography.hazmat.primitives import hashes

#plain_text = 'Mallory has modified the message'

print('Message = ' + plain_text)
digest = hashes.Hash( hashes.SHA1() )
baMessage = plain_text.encode('utf-8')
digest.update(baMessage)
bob_hashFromMessage_bytes = digest.finalize()
print_bytes('Bob: h =  hash(message) 0x', bob_hashFromMessage_bytes, 16)

bob_hashFromSignature_long  = pow(alice_rsa_signature, e, n)
bob_hashFromSignature_bytes = long_to_bytes(bob_hashFromSignature_long)

print_bytes('Bob: h = s^e (mod n) = 0x', bob_hashFromSignature_bytes, 16)

print("Signature valid:", bob_hashFromMessage_bytes == bob_hashFromSignature_bytes)

Message = Hello World
Bob: h =  hash(message) 0x0A 4D 55 A8 D7 78 E5 02 2F AB 70 19 77 C5 D8 40 
                          BB C4 86 D0 
Bob: h = s^e (mod n) = 0x0A 4D 55 A8 D7 78 E5 02 2F AB 70 19 77 C5 D8 40 
                         BB C4 86 D0 
Signature valid: True
